In [21]:
import fitz
import os
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.http import models

# Extraction knowledge

In [15]:
folder_path = "E:/folder kuliah/Project/DATATHON/agriculture-ai-powered/Knowledge"  # Ganti dengan path folder Anda
pdf_pages_dict = {}

for filename in os.listdir(folder_path):
    if filename.endswith(".pdf"):
        pdf_path = os.path.join(folder_path, filename)
        doc = fitz.open(pdf_path)
        pages_text = []
        for page in doc:
            text = page.get_text()
            pages_text.append(text)
        doc.close()
        pdf_pages_dict[filename] = pages_text

# add to Qdrant 

In [10]:
def load_books(folder_path, namefile):
    # Gabungkan folder path dan nama file
    file_path = os.path.join(folder_path, namefile)
    with open(file_path, 'r', encoding='utf-8') as file:
        isi = file.read()
    return isi

In [12]:
# load books 
Soil = load_books("E:/folder kuliah/Project/DATATHON/agriculture-ai-powered/Knowledge", "Building-Soils-for-Better-Crops.txt")
nutrients = load_books("E:/folder kuliah/Project/DATATHON/agriculture-ai-powered/Knowledge", "NUTRIENT INTERACTIONS IN CROP PLANTS.txt")
weather = load_books("E:/folder kuliah/Project/DATATHON/agriculture-ai-powered/Knowledge", "Weather and Management Impact on Crop Yield Variability in Rotations.txt")

In [20]:
# --- Membuat Embedding ---
model = SentenceTransformer("all-MiniLM-L6-v2")

# Siapkan data untuk Qdrant
all_texts = []
all_payloads = []
for pdf_name, pages in pdf_pages_dict.items():
    for i, page_text in enumerate(pages):
        all_texts.append(page_text)
        all_payloads.append({
            "pdf_name": pdf_name,
            "page_number": i + 1,
            "text": page_text
        })

embeddings = model.encode(all_texts)

# --- Koneksi ke Qdrant dan Add Data ---
client = QdrantClient(host="localhost", port=6333)

# Buat collection baru (hanya perlu sekali)
client.recreate_collection(
    collection_name="pdf_knowledge",
    vectors_config=models.VectorParams(
        size=embeddings.shape[1],
        distance=models.Distance.COSINE
    ),
)

# Siapkan points untuk upsert
points = [
    models.PointStruct(
        id=i,
        vector=embeddings[i].tolist(),
        payload=all_payloads[i]
    )
    for i in range(len(all_texts))
]

# Add ke Qdrant
client.upsert(
    collection_name="pdf_knowledge",
    points=points
)

print("Semua data berhasil dimasukkan ke Qdrant.")


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
C:\Users\Tpny\AppData\Local\Temp\ipykernel_16892\3256267820.py:22: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


Semua data berhasil dimasukkan ke Qdrant.
